In [177]:
import altair as alt
from vega_datasets import data
import json
import pandas as pd
import numpy as np

In [2]:
# Example of tooltip plot over map

airports = data.airports()
states = alt.topo_feature(data.us_10m.url, feature='states')

# US states background
background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=500,
    height=300
).project('albersUsa')

# airport positions on background
points = alt.Chart(airports).mark_circle(
    size=10,
    color='steelblue'
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    tooltip=['name', 'city', 'state']
)

background + points

alt.LayerChart(...)

In [3]:
# Opening JSON file
f = open(r'../data/maps/county003wtown.json',)
 
# returns JSON object as a dictionary
mass_json = json.load(f)
    
# Closing file
f.close()


In [4]:
# reading dictionary into altair data format
mass_data = alt.Data(values=mass_json)

In [5]:
# Map of Williamstown
background = alt.Chart(mass_data).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=500,
    height=900
).project('albersUsa')

background

alt.Chart(...)

In [6]:
# Opening JSON file
froad = open(r'../data/maps/roads.json',)
 
# returns JSON object as a dictionary
wtown_roads = json.load(froad)
    
# Closing file
froad.close()

# reading dictionary into altair data format
wtown_roads_data = alt.Data(values=wtown_roads)

In [204]:
# Opening JSON file
# Opening JSON file
fpts = open(r'../data/maps/points.json',)
 
# returns JSON object as a dictionary
wtown_points = json.load(fpts)


# Closing file
froad.close()

# read data into nice dataframe
df = pd.json_normalize(wtown_points['features'])
coords = pd.DataFrame(df["geometry.coordinates"].to_list(), columns=['lat', 'lon'])
props = df[["properties.type","properties.name"]]
props = props.rename(columns={"properties.type": "type", "properties.name": "name"}, errors="raise")

props.replace("", np.nan, inplace=True)
props.dropna(subset = ["name"], inplace=True)
props.dropna(subset = ["type"], inplace=True)

points_data = pd.merge(coords, props , how="outer", left_index = True, right_index=True)
points_data.replace("", np.nan, inplace=True)
points_data.dropna(inplace=True)

# reading dictionary into altair data format
wtown_pts_data = alt.Data(values=points_data)

points_data.head()

lat        lon       type                            name
0  -73.193175  42.711363    parking    Lehovec Natural Area Parking
3  -73.205100  42.655474  trailhead            Haley Farm Trailhead
9  -73.206583  42.734638     police  Williamstown Police Department
10 -73.202392  42.713540    library        Chapin Rare Book Library
11 -73.214019  42.707049    library         Manton Research Library

In [210]:
# Map of Williamstown
background = alt.Chart(mass_data).mark_geoshape(
    fill='lightgray',
    stroke='none'
).properties(
    width=1500,
    height=1350
).project('mercator')

road_layer = alt.Chart(wtown_roads_data).mark_geoshape(
    fill='none',
    stroke='black'
).properties(
    width=1500,
    height=1350
).project('mercator')

points_layer = alt.Chart(points_data).mark_circle(
    size=100,
    color='steelblue'
).encode(
    longitude='lat:Q',
    latitude='lon:Q',
    tooltip=['name','type']
)


background + road_layer + points_layer


alt.LayerChart(...)

In [201]:
topo_data = alt.topo_feature(data.world_110m.url, 'countries')

slider_scale = alt.binding_range(min=50, max=1000, step=10)

select_scale = alt.selection_single(name="zoom", fields=['scale'],
                                   bind=slider_scale, init={'scale': 50})

alt.Chart(topo_data).mark_geoshape().project(
    scale=200
).add_selection(select_scale)

alt.Chart(...)